In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder, StandardScaler
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report

In [6]:
# import data
df = pd.read_csv('/Users/rachanon.cho/Documents/Kaggle Practice/California Wildfires/data/cal_wildfire.csv')

/var/folders/_1/nrm1y7js22bdlyxkt1mzg_p40000gn/T/ipykernel_89566/3537204748.py:2: DtypeWarning: Columns (13,37,38) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/Users/rachanon.cho/Documents/Kaggle Practice/California Wildfires/data/cal_wildfire.csv')


In [7]:
# Define the target variable
TARGET_COLUMN = '* Damage'

# Remove 'Inaccessible' entries from target variable
df = df[df[TARGET_COLUMN] != 'Inaccessible']

# Merge similar damage categories
df[TARGET_COLUMN] = df[TARGET_COLUMN].replace(
    ['Minor (10-25%)', 'Major (26-50%)', 'Affected (1-9%)'], 'Affected')
df[TARGET_COLUMN] = df[TARGET_COLUMN].replace(['Destroyed (>50%)'], 'Destroyed')


In [9]:
# Encoding target variable (convert labels to numbers)
label_encoder = LabelEncoder()
df[TARGET_COLUMN] = label_encoder.fit_transform(df[TARGET_COLUMN])

# Select relevant features (modify as needed)
FEATURE_COLUMNS = ['# Units in Structure (if multi unit)', 'Assessed Improved Value (parcel)', 'Latitude', 'Longitude']
df = df[FEATURE_COLUMNS + [TARGET_COLUMN]].dropna()

In [10]:
# Split dataset into train and test sets
X = df[FEATURE_COLUMNS]
y = df[TARGET_COLUMN]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


In [11]:
# Apply SMOTE to balance classes
smote = SMOTE(random_state=42)
X_train, y_train = smote.fit_resample(X_train, y_train)

In [12]:
# Standardize numerical features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [13]:
# Train Random Forest Classifier
# Train XGBoost Classifier
xgb_model = XGBClassifier(objective='multi:softmax', eval_metric='mlogloss', use_label_encoder=False)
xgb_model.fit(X_train, y_train)

# Make predictions
y_pred = xgb_model.predict(X_test)

/Users/rachanon.cho/opt/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


In [15]:
# Evaluate model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy:.4f}")
print("Classification Report:")
print(report)

Accuracy: 0.8313
Classification Report:
              precision    recall  f1-score   support

           0       0.18      0.23      0.20       212
           1       0.81      0.80      0.81      2062
           2       0.89      0.88      0.89      3601

    accuracy                           0.83      5875
   macro avg       0.63      0.64      0.63      5875
weighted avg       0.84      0.83      0.84      5875

